<h1>Extracting and Visualizing Stock Data</h1>



In [20]:
!pip install beautifulsoup4==4.10.0
!pip install nbformat==5.10.2

In [21]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [22]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

##Graphing Function


In [23]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

##Using yfinance to Extract Stock Data


In [24]:
tesla = yf.Ticker("TSLA")

In [25]:
tesla_data = tesla.history(period="max")

Failed to get ticker 'TSLA' reason: Expecting value: line 1 column 1 (char 0)
- TSLA: No timezone found, symbol may be delisted


In [26]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume


##Webscraping to Extract Tesla Revenue Data


In [27]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

In [28]:
soup = BeautifulSoup(html_data,'html.parser')

In [29]:
read_html_pandas_data = pd.read_html(url)
read_html_pandas_data = pd.read_html(str(soup))
tesla_revenue = read_html_pandas_data[1]

tesla_revenue.columns = ['Date','Revenue']#to rename columns


Removing the comma and dollar sign from the `Revenue` column.


In [30]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].astype(str).str.replace('$', '').str.replace(',', '').replace('nan', pd.NA)
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors='coerce')

Removing all null or empty strings in the Revenue column.


In [31]:
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [32]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31.0
49,2010-06-30,28.0
50,2010-03-31,21.0
52,2009-09-30,46.0
53,2009-06-30,27.0


## Using yfinance to Extract Stock Data


In [33]:
game = yf.Ticker("GME")

Using the ticker object and the function `history` to extract stock information and save it in a dataframe named `gme_data`. I set the `period` parameter to `max` so we get information for the maximum amount of time.


In [34]:
gme_data = game.history(period="max")

Failed to get ticker 'GME' reason: Expecting value: line 1 column 1 (char 0)
- GME: No timezone found, symbol may be delisted


In [35]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume


## Webscraping to Extract GME Revenue Data


In [36]:
ulr = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data = requests.get(ulr).text

In [37]:
soup = BeautifulSoup(html_data, "html.parser")

In [38]:
gme_revenue = pd.DataFrame(columns = ["Date", "Revenue"])
for row in soup.find_all("tbody")[1].find_all("tr"):
    col = row.find_all("td")
    date= col[0].text
    rev = col[1].text
    new_row = pd.DataFrame({"Date":[date], "Revenue":[rev]})
    gme_revenue = pd.concat([gme_revenue, new_row], ignore_index=True)
gme_revenue["Revenue"] = gme_revenue['Revenue'].astype(str).str.replace('$', '').str.replace(',', '')
gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"])

In [39]:
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


## Plotting Tesla Stock Graph


In [40]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

## GameStop Stock Graph


In [41]:
make_graph(gme_data, gme_revenue, 'GameStop')